## Classification of Speech Frames

### PART IV:  RECOGNIZE A PHONEME FROM FILTERBANK FEATURES WITH A DNN

### 1. Setting up your Python Environment

1. Import Python's Machine Learning Stack and other utilities

2. Quite a few helper routines are defined for learning our Neural Network in Pytorch. There is no need to go through this in detail, but interested people can of course do this. There are 3 blocks of code:

  a. Auxiliary functions: data loading/saving + feature extraction + phone label mapping

  b. Pytorch functions: NN architecture + training + evaluation 

  c. Define setups: predefined setups (feature extraction, labels, training data)




In [1]:
#@title Import libraries

%matplotlib inline
import sys,os,io
import math
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import scipy.io as sio
import urllib.request

# imports from pytorch
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

# reproducibility 
torch.manual_seed(0) 
np.random.seed(0)

# imports from the scikit-learn 
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import Perceptron
from sklearn import metrics as skmetrics
from sklearn.mixture import GaussianMixture
from scipy.fftpack import dct

try:
  import google.colab
  IN_COLAB = True
  ! pip install git+https://github.com/compi1234/pyspch.git
except:
  IN_COLAB = False

from pyspch.GaussianMixtureClf import GaussianMixtureClf
import pyspch.display as spchd

  Cloning https://github.com/compi1234/pyspch.git to /tmp/pip-req-build-n88wky35
  Running command git clone -q https://github.com/compi1234/pyspch.git /tmp/pip-req-build-n88wky35
  Created wheel for pyspch: filename=pyspch-0.4.1-py3-none-any.whl size=728011 sha256=771a78a9d261a3bf8f3906e3004b09dc27dc047fd8252092dbbc598ee884295f
  Stored in directory: /tmp/pip-ephem-wheel-cache-14to1omk/wheels/37/48/d3/ecbc5875edffd413745fe137437cdfebe588a3722fb8efc4d7
Successfully built pyspch


In [2]:
#@title Auxiliary functions 

## Pickle functions 
import pickle
import gzip

# save pickled file
def pickle_data(df, filename):   
    file_object = gzip.open(filename, "wb")
    print("Generating " + filename, end=" ...")
    pickle.dump(df, file_object)
    print("done")
    file_object.close()

# open pickled file
def unpickle_data(filename):
    file_object = open(filename, "rb")
    df = pickle.load(file_object)
    file_object.close()
    return df


## Download functions

import requests

# download from url and write to file
def write_from_url(url, filename):
    r = requests.get(url)
    with open(filename, 'wb') as f:
        f.write(r.content)

## Phoneme segmentation  

# phone segmentation to phone per frame in utterance
def phnseg_to_phns(phnseg, nframes):
    phn_list = []
    for seg_idx, seg in phnseg.iterrows():
        seg_nframes = seg['t1'] - seg['t0']
        phn_list += [seg['seg']] * seg_nframes
        
    return phn_list    
 
# phone segmentation to center frame of phone per frame
def phnseg_to_center_frames(phnseg, nframes):
    center_frame_ids = []
    for seg_idx, seg in phnseg.iterrows():
        seg_nframes = seg['t1'] - seg['t0']
        center_frame_ids += [int(seg['t0'] + seg_nframes / 2)] * seg_nframes
        
    return center_frame_ids

##  Window

# The window is determined by utterance boundaries (frame_idx, utt_nframes) 
# and the window parameters (window_nframes, stride). The window is represented 
# by indices relative to the frame index.

# window indices relative to frame index
def get_window(frame_idx, utt_nframes, window_nframes, stride, shift=0):
    
    def window_bounds(window_nframes):
        if window_nframes % 2 == 0:    
            # even case: start-i-center-i-i-end (example window_nframes=6)       
            start = -window_nframes / 2 + 1
            end = window_nframes / 2
        else:                       
            # uneven case: start-i-ref-i-stop (example window_nframes=5)       
            start = -(window_nframes - 1) / 2
            end = (window_nframes - 1) / 2  
            
        return start, end
    
    def adjust_window_to_utt_bounds(window):
        new_window = []
        for rel_idx in window:
            rel_idx = max(rel_idx, 0 - frame_idx)
            rel_idx = min(rel_idx, utt_nframes - 1 - frame_idx)
            new_window.append(rel_idx)
            
        return np.array(new_window)
    
    # window boundaries w.r.t. center frame
    start, end = window_bounds(window_nframes)
    # window of evenly spaced frames around center frame
    window = np.arange(start, end + 1).astype(int) 
    # apply stride and shift
    window = window * stride + shift 
    # adjust window indices out of the utterance bounds
    window = adjust_window_to_utt_bounds(window)
    
    return window

# window indices for each frame in utterance
def get_windows_for_utt(utt_nframes, window_nframes, stride, shift):
    utt_windows = []
    # iterate over frames in utterance
    for frame_idx in range(utt_nframes):
        window = get_window(frame_idx, utt_nframes,
                            window_nframes, stride, shift)
        utt_windows.append(window)
    
    return utt_windows

# apply window to frames  
def apply_window(frames, frame_ids, frame_windows):
    inputs = []
    # ensure type is numpy for easy indexing
    frames = np.array(frames)
    # iterate over frames 
    for frame_idx, frame_window in zip(frame_ids, frame_windows):
        # apply window
        feature = np.stack(frames[frame_window + int(frame_idx)], axis=0)
        # save feature
        inputs.append(feature)
        
    return np.stack(inputs, axis=0)

## Reshape

# reshape features assuming first dimension is 'examples'
def reshape_features(frames, shape=(-1,)):
    # n_examples = len(frames)
    # new_shape = (n_examples, *shape)
    
    return [np.reshape(frame, shape) for frame in frames]

## Normalization and derrivatives

# normalization
def normalize_data(data, axis=0, normMean=False, normVar=True):
    if normMean:
        mean = data.std(axis=axis) 
        data = np.subtract(data, mean)
    if normVar:
        stdev = data.std(axis=axis) 
        data = np.divide(data, stdev)
    
    return data

# temporal derrivatives
def add_derrivatives_data(data, delta=True, ddelta=True):   
    if delta:
        data_delta = np.gradient(data, axis=0)
        data = np.concatenate((data, data_delta), axis=-1)
    if ddelta:
        data_delta = np.gradient(data, axis=0)
        data_ddelta = np.gradient(data_delta, axis=0)
        data = np.concatenate((data, data_delta, data_ddelta), axis=-1)
 
    return data


## Wrapper for assembling dataframe containing frames 

# assemble dataframe
#   = row per frame
# - path                    = utterance identifier
# - center frame index      = index of center frame for current phoneme
# - frame index             = index of frame for current phoneme
# - window indices          = window indices 
def assemble_frame_dataframe(df, window_nframes=5, stride=2, shift=0,
                             delta=True, ddelta=False,
                             normMean=False, normVar=True):
    frame_dfs = []
    # iterate over utterances
    for _, utt in df.iterrows():
        
        ## modify feature vector
        # utterance data
        data = utt['data']
        # add temporal derrivatives
        data = add_derrivatives_data(data, delta=delta, ddelta=ddelta)
        # normalize standard deviation per channel (feature dimension)
        data = normalize_data(data, axis=0, normMean=normMean, normVar=normVar)
        
        ## labels and names
        # number of frames in utterance
        utt_nframes = data.shape[0]
        # utterance name for each frame
        utt_names = [utt['utt_name']] * utt_nframes
        # phone segmentation to phone per frame in utterance
        phnseg = utt['phn_seg']
        utt_phns = phnseg_to_phns(phnseg, utt_nframes)
        centre_frame_ids = phnseg_to_center_frames(phnseg, utt_nframes)
        # - indices of (feature) window per frame in utterance 
        utt_windows = get_windows_for_utt(utt_nframes, window_nframes=window_nframes, 
                                          stride=stride, shift=shift)
        
        # index of last frame
        # - correct for occasional mismatch in number of frames and phone labels
        eidx = min(utt_nframes, len(utt_phns))
        
        ## frame dataframe per utterance  
        # - row per frame in utterance
        # - columns: "utt_name", "frame_idx", window", "data", "phn_label"
        frame_df = pd.DataFrame({'utt_name' : utt_names[:eidx],
                               'center_frame_idx' : centre_frame_ids[:eidx],
                               'frame_idx' : np.arange(utt_nframes)[:eidx],
                               'window' :  utt_windows[:eidx],
                               'data' : list(data[:eidx]),
                               'org_phn_label' : utt_phns[:eidx]})
         
        frame_dfs.append(frame_df)
    
    # concatenate frame dataframes per utterance to final frame dataframe 
    frame_df = pd.concat(frame_dfs, ignore_index=True)
    
    return frame_df


## Phone and label mapping 

# TIMIT phone label to phone label (using phone_map_file)
def get_phn2phn_map(phone_map_file, ocol, icol=0):
    phn2phn = {}
    with open(phone_map_file) as file:
        for line in file:
            w = line.strip().split("\t")
            phn2phn[w[icol]] = w[ocol]
            
    return phn2phn 

# phone label to numeric label
def get_phn2lab_map(phn_labels):
    phn_labels = unique_list(phn_labels)
    phn2lab_map = {phn : i for i, phn in enumerate(phn_labels) }
    lab2phn_map = {i : phn for i, phn in enumerate(phn_labels) }
    
    return phn2lab_map, lab2phn_map 

# numeric label (based on 'icol') to other numeric label (based on '0col')
def get_lab2lab_dict(phone_map_file, icol, ocol, phn_set=None):
    # phn2phn
    iphn2ophn = get_phn2phn_map(phone_map_file, ocol, icol)

    # phn2lab x2
    iphn2lab, _ = get_phn2lab_map(list(iphn2ophn.keys()))
    ophn2lab, _ = get_phn2lab_map(list(iphn2ophn.values())) 

    # lab2lab
    ilab2olab = {iphn2lab[k]:ophn2lab[v] for k,v in iphn2ophn.items()}
    
    return ilab2olab

# get unique items in list
def unique_list(lst): 
    # dict perserves order (vs. set)
    return list(dict.fromkeys(lst))


## Operations on dataframe  {display-mode: "form"}

# split 'utt_name' column for easy data selection
def split_string_column(df, col_dict, sep="/"):  
    for key, value in col_dict['target'].items():
        df[key] = df.apply(lambda row: row[col_dict['source']].split(sep)[value], axis=1)
    
    return df 

# data selection based on regex in columns
def filter_regex_dataframe(df, include, exclude=None):
    if include is not None:
        for key, value in include.items():
            if value is not None:
                df = df[df[key].str.contains(value)]
                
    if exclude is not None:
        for key, value in exclude.items():
            if value is not None:
                df = df[~df[key].str.contains(value)]
    
    return df

### Wrapper functions

def phone_and_label_mapping(phone_map_file, ocol=1, phn_set=None):

    # phone mapping parameters
    # ocol -> {0 : TIMIT 61 phone label, 1 : 48 phone label, 2: 39 phone label}

    # phone mapping - phn2phn dictionairy = {TIMIT phone label : phone label}
    phn2phn = get_phn2phn_map(phone_map_file, ocol=ocol)

    # label mapping - phn2lab dictionairy = {phone label : integer label}
    if phn_set == None:
        phn_set = unique_list([phn for phn in phn2phn.values()]) # all values
    
    phn2lab, lab2phn = get_phn2lab_map(phn_set)

    return phn2phn, phn2lab, lab2phn, phn_set

# split in train / validation / test set 
def utterance_based_data_split(frames_df): 

    # add columns for easy splitting
    col_dict = {'source' : 'utt_name',
                'target' : {'dataset' : 0, 'dialect' : 1, 
                            'speaker' : 2, 'utterance' : 3}}
    frames_df = split_string_column(frames_df, col_dict=col_dict, sep='/')

    # define split
    train_re_incl = {'dataset' : 'train'}
    train_re_excl = {'dialect' : 'dr8', 'utterance' : 'sa'}
    valid_re_incl = {'dataset' : 'train', 'dialect' : 'dr8'}
    valid_re_excl = {'utterance' : 'sa'}
    test_re_incl = {'dataset' : 'test'}
    test_re_excl = {'utterance' : 'sa'}

    # split
    train_df = filter_regex_dataframe(frames_df, train_re_incl, train_re_excl)
    valid_df = filter_regex_dataframe(frames_df, valid_re_incl, valid_re_excl)
    test_df = filter_regex_dataframe(frames_df, test_re_incl, test_re_excl)

    return train_df, valid_df, test_df



In [3]:
#@title Pytorch functions

### Neural network architecture ###

# simple fully-connected feedforward neural network 
class Dnn(nn.Module):
    
    def __init__(self, in_dim, out_dim, hidden_layer_sizes):
        super(Dnn, self).__init__()

        # attributes
        self.in_dim = in_dim
        self.out_dim = out_dim
        self.hidden_layer_sizes = hidden_layer_sizes

        # parameters
        layer_sizes = (in_dim, *hidden_layer_sizes, out_dim)
        layer_sizes_pairwise = [(layer_sizes[i], layer_sizes[i+1]) for 
                                 i in range(len(layer_sizes)-1)]

        # define architecture
        modulelist = nn.ModuleList([])  
        for i, (layer_in_size, layer_out_size) in enumerate(layer_sizes_pairwise):
            modulelist.append(nn.Linear(layer_in_size, layer_out_size))
            if i < len(self.hidden_layer_sizes):
                modulelist.append(nn.Sigmoid())

        # define network as nn.Sequential
        self.net = nn.Sequential(*modulelist)

    def forward(self, x):
        x = self.net(x)
        return x

class View(nn.Module):
    def __init__(self, shape):
        super().__init__()
        self.shape = shape

    def forward(self, input):
        batch_size = input.size(0)
        shape = (batch_size, *self.shape)
        out = input.view(shape)
        return out


# fully-connected neural network with 1D convolutional layer
class Cnn1d(nn.Module):
    
    def __init__(self, in_dim, out_dim, hidden_layer_sizes,
                 kernel_size=(5, ), padding=(2, ), n_filters=100,
                 cnn_position=0):
        super(Cnn1d, self).__init__()

        # attributes
        self.in_dim = in_dim
        self.out_dim = out_dim
        self.hidden_layer_sizes = hidden_layer_sizes

        # parameters
        layer_sizes = (in_dim, *hidden_layer_sizes, out_dim)
        layer_sizes_pairwise = [(layer_sizes[i], layer_sizes[i+1]) for 
                                 i in range(len(layer_sizes)-1)]

        # define architecture
        modulelist = nn.ModuleList([])
        for i, (layer_in_size, layer_out_size) in enumerate(layer_sizes_pairwise):

            if i == cnn_position:
                modulelist.append(nn.Conv1d(layer_in_size, n_filters, 
                                            kernel_size, padding=padding))
                modulelist.append(View(shape=(-1, )))
                    
            else:
                modulelist.append(nn.Linear(layer_in_size, layer_out_size))

            if i < len(self.hidden_layer_sizes):
                modulelist.append(nn.Sigmoid())

        # define network as nn.Sequential
        self.net = nn.Sequential(*modulelist)

    def forward(self, x):
        x = self.net(x)
        return x

def init_weights(m, bias=0.00):
    if type(m) == nn.Linear:
        torch.nn.init.xavier_uniform_(m.weight)
        m.bias.data.fill_(bias)


# fully-connected neural network with 1D convolutional layer
class Cnn2d(nn.Module):
    
    def __init__(self, in_dim, out_dim, hidden_layer_sizes,
                 kernel_size=(5, 5), padding=(2, 2), n_filters=25,
                 cnn_position=0):
        super(Cnn2d, self).__init__()

        # attributes
        self.in_dim = in_dim
        self.out_dim = out_dim
        self.hidden_layer_sizes = hidden_layer_sizes

        # parameters
        layer_sizes = (in_dim, *hidden_layer_sizes, out_dim)
        layer_sizes_pairwise = [(layer_sizes[i], layer_sizes[i+1]) for 
                                 i in range(len(layer_sizes)-1)]

        # define architecture
        modulelist = nn.ModuleList([])
        for i, (layer_in_size, layer_out_size) in enumerate(layer_sizes_pairwise):

            if i == cnn_position:
                modulelist.append(nn.Conv2d(layer_in_size, n_filters, 
                                            kernel_size, padding=padding))
                modulelist.append(View(shape=(-1, )))
                    
            else:
                modulelist.append(nn.Linear(layer_in_size, layer_out_size))

            if i < len(self.hidden_layer_sizes):
                modulelist.append(nn.Sigmoid())

        # define network as nn.Sequential
        self.net = nn.Sequential(*modulelist)

    def forward(self, x):
        x = self.net(x)
        return x

### Neural network training ###

# simple batch gradient descent 
def train_batch(network, train_X, train_y, criterion, optimizer, device, 
                n_epochs=500, every=10):

    # data-format is torch tensor + send to device (ex. GPU)
    if not torch.is_tensor(train_X):
        train_X = torch.tensor(train_X, dtype=torch.float32).to(device)
    if not torch.is_tensor(train_y):
        train_y = torch.tensor(train_y, dtype=torch.long).to(device)

    # send network to device (ex. GPU)
    network.to(device)

    # set network to training mode (vs. evaluation mode)
    network.train() 

    # save training loss
    train_loss = []

    # train for some epochs - full batch
    for epoch in range(n_epochs):
        optimizer.zero_grad()
        output = network(train_X)
        loss = criterion(output, train_y)
        loss.backward()
        optimizer.step()
        if epoch%every == 0:
          print('Epoch {}, loss {}'.format(epoch, loss.item()))
        
        train_loss.append(loss.item())
      
    return train_loss
            
# mini-batch gradient descent with early stopping
def train(network, train_dl, criterion, optimizer, device, n_epochs=500, 
          valid_X=None, valid_y=None, patience=5, every=10, 
          current_epoch=0):

    # set early stopping counter
    cnt_valid_loss_increase = 0

    # current_epoch
    last_epoch = current_epoch

    # data-format is torch tensor + send to device (ex. GPU)
    if not torch.is_tensor(valid_X):
        valid_X = torch.tensor(valid_X, dtype=torch.float32).to(device)
    if not torch.is_tensor(valid_y):
        valid_y = torch.tensor(valid_y, dtype=torch.long).to(device)

    # save training and validation loss
    train_loss_list = []
    valid_loss_list = []

    #  per epoch: update network parameters 
    for epoch in range(n_epochs):

        # network to training mode
        network.train()
        
        # early stopping 
        earlystoppping = cnt_valid_loss_increase > patience
        if earlystoppping:
            print("stopped early after %d epochs" % (epoch))
            return train_loss_list, valid_loss_list, last_epoch

        # per mini-batch: compute gradient (backpropagation) + take step 
        running_loss = 0
        steps = 0 
        for i, data in enumerate(train_dl, 0):

            # mini-batch inputs and labels
            frames, labels = data  
            
            # zero the parameter gradients
            optimizer.zero_grad()
            
            # forward + backward + optimize 
            outputs = network.net(frames)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()  

            # running loss
            running_loss += loss.item()

        # training loss
        mean_minibatch_loss = running_loss/len(train_dl)
        train_loss_list.append(mean_minibatch_loss)
        if epoch%every == 0:   
            print("Epoch %d -- av. loss per mini-batch %.2f" % (epoch, mean_minibatch_loss))

        # validation loss
        if valid_X is not None and valid_y is not None: 
            # network to evaluation mode
            network.eval()

            with torch.no_grad():
                valid_outputs = network.net(valid_X)
                valid_loss = criterion(valid_outputs, valid_y)
                valid_loss_list.append(valid_loss.item())
                
                # early stoppping
                loss_increase = valid_loss_list[-1] > min(valid_loss_list[-patience:])
                if loss_increase:
                    # no improvement compared to last 'patience' steps
                    cnt_valid_loss_increase += 1
                else:
                    cnt_valid_loss_increase = 0

        last_epoch = current_epoch + epoch + 1

    return train_loss_list, valid_loss_list, last_epoch


### Neural network saving and loading ###

# save model
def save_model(epoch, model, criterion, optimizer, model_file):
    torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'criterion': criterion,
            'optimizer_state_dict': optimizer.state_dict(),
            }, model_file)
    
    doPrint = False
    if doPrint:
        print('Saved model to: ' + model_file)

# load model
def load_model(model, optimizer, model_file, device, strict=False):
    doPrint = False
    if doPrint:
        print('Loading: ' + model_file)
        
    checkpoint = torch.load(model_file, map_location=device)
    epoch = checkpoint['epoch']    
    model.load_state_dict(checkpoint['model_state_dict'], strict=strict)
    criterion = checkpoint['criterion']
    if doPrint:
        print(checkpoint['optimizer_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    
    return epoch, model, criterion, optimizer

### Neural network evaluation ###

# evaluate criterion
def evaluate_criterion(network, test_X, test_y, criterion, device):
    
    # data-format is torch tensor + send to device (ex. GPU)
    if not torch.is_tensor(test_X):
        test_X = torch.tensor(test_X, dtype=torch.float32).to(device)
    if not torch.is_tensor(test_y):
        test_y = torch.tensor(test_y, dtype=torch.long).to(device)

    # send network to device (ex. GPU)
    network.to(device)

    # set network to evaluation mode (vs. training mode)
    network.eval() 

    # compute loss based on criterion
    pred_y = network.net(test_X)
    loss = criterion(pred_y, test_y)

    return loss.item()

# evaluate confusion matrix
def confusionmatrix(network, test_X, test_y, device, 
                    n_per_pass=2**10, unsqueeze=False, unsqueeze2d=False):
    
    # data-format is torch tensor + send to device (ex. GPU)
    if not torch.is_tensor(test_X):
        test_X = torch.tensor(test_X, dtype=torch.float32).to(device)
    if not torch.is_tensor(test_y):
        test_y = torch.tensor(test_y, dtype=torch.long).to(device)

    # send network to device (ex. GPU)
    network.to(device)

    # set network to evaluation mode (vs. training mode)
    network.eval() 

    # compute confusion matrix
    cm = np.zeros((network.out_dim, network.out_dim))
        
    # splitting up dataframe in managable chunks
    n_samples = len(test_X)
    loop_n_times = math.ceil(n_samples/n_per_pass)
    for i in range(loop_n_times):
      test_X_part = test_X[i*n_per_pass:(i+1)*n_per_pass]
      test_y_part = test_y[i*n_per_pass:(i+1)*n_per_pass]
      for input, label in zip(test_X_part, test_y_part):
        if unsqueeze: prob = network.net(torch.unsqueeze(input, 0))
        elif unsqueeze2d: prob = network.net(torch.reshape(input, (1, *input.shape)))
        else: prob = network.net(input) # output = posterior class probabilities
        pred = torch.argmax(prob) # prediction = label (or neuron) with highest probability (One-Hot Encoding)
        cm[label][pred] += 1
    
    return cm.astype(int)


# remap confusion matrix
def remap_confusionmatrix(confusionmatrix, lab2lab_dict):
    
    n_classes = confusionmatrix.shape[0]
    n_classes_new = len(np.unique((np.fromiter(lab2lab_dict.values(), dtype=int))))
    confusionmatrix_new = np.zeros((n_classes_new, n_classes_new))
    
    # remap classes
    for row in range(n_classes):
        for col in range(n_classes):
            confusionmatrix_new[lab2lab_dict[row], lab2lab_dict[col]] += confusionmatrix[row,col]
            
    return confusionmatrix_new

# evaluate phone error rate
def evaluate_PER(confusionmatrix):

    n_classes = confusionmatrix.shape[0]    
    
    # compute ER 
    trace = np.trace(confusionmatrix)
    ER = 1- trace.sum() / confusionmatrix.sum()
        
    # compute ER per class (disregarding non label or not)
    no_examples_pc = confusionmatrix.sum(axis=1)
    ER_pc = [None] * n_classes
    for i in range(n_classes):
        if no_examples_pc[i] != 0:
            ER_pc[i] = 1-confusionmatrix[i,i] / (no_examples_pc[i])
        
    return ER, ER_pc


### Confusion matrix plot ###

import matplotlib.pyplot as plt
import seaborn as sns

# pretty printing
def plot_confusion_matrix(cm, labels=[], cmap=[], figsize=(6,6), fontsize=14):

    if len(labels) == 0:
        df_cm = pd.DataFrame(cm)
    else:
        df_cm = pd.DataFrame(cm, labels, labels)
        
    f, ax = plt.subplots(figsize=figsize)
    sns.set(font_scale=1.4)#for label size
    sns.heatmap(df_cm, annot=True, fmt="d", annot_kws={"fontsize": fontsize,"color":'k'},
                square=True, linecolor='k', linewidth=1.5, cmap=cmap, cbar=False)
    ax.tick_params(axis='y', labelrotation=0.0, left=True)
    # font size
    plt.title('Confusion matrix')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.show()

### Dataset ###

class SimpleDataset(Dataset):
    """Simple dataset for easy sampling."""

    def __init__(self, data_X, data_y, labels, labeldict, device):

        # dimensionality
        self.n_samples, self.n_features = data_X.shape[0:2]
        self.n_classes = len(labels)

        # input data
        self.frames = data_X # (n_samples, n_features)
        self.frames = torch.as_tensor(self.frames, dtype=torch.float32).to(device)

        # labels
        if data_y.dtype != "int64":
            data_y = np.vectorize(labeldict.get)(data_y)
        self.labels = torch.as_tensor(data_y, dtype=torch.long).to(device)

    def __len__(self):
        
        return self.n_samples

    def __getitem__(self, idx):
        frame = self.frames[idx] 
        label = self.labels[idx]
        
        return frame, label


In [4]:
#@title Define setups

def prepare_baseline(utt_df, utt_names=None, phone_map_file='phones-61-48-39.txt'):
    
    ## DATA SELECTION -- utterance based (speed up)
    if utt_names is not None:
        utt_df = utt_df[utt_df['utt_name'].isin(utt_names)]

    ## FEATURE EXTRACTION

    # input feature parameters
    window_nframes = 11
    stride = 2
    shift = 0
    delta, ddelta = False, False
    normMean, normVar = False, True

    # generate dataframe containing frames
    frames_df = assemble_frame_dataframe(utt_df, window_nframes, stride, shift, \
                                        delta, ddelta, normMean, normVar)

    # assemble input feature from window
    features = apply_window(frames_df['data'], frames_df.index, frames_df['window'])
    #frames_df['feature'] = features
    frames_df['feature'] = reshape_features(features, (-1,)) 

    ## LABELS

    # phone and label mapping
    ocol = 1 # {'61' : 0, '48' : 1, '39' : 2}
    phn_set = None
    phn2phn, phn2lab, lab2phn, phn_set = phone_and_label_mapping(phone_map_file, ocol, phn_set)

    # map phones and labels
    frames_df['phn_label'] = frames_df['org_phn_label'].map(phn2phn) # TIMIT phone label -> phone label
    frames_df['int_label'] = frames_df['phn_label'].map(phn2lab)  # phone label ->  integer label

    ## DATA SELECTION -- frame based

    # drop phones not in phn_set
    frames_df = frames_df[~frames_df['int_label'].isna()] 

    # drop phones with large shift in phn_set
    doShift = False
    max_shift = 5 # shift = distance to center frame
    if doShift:  
      frames_df['shift'] = frames_df['frame_idx'] - frames_df['center_frame_idx']
      frames_df = frames_df[frames_df['shift'].abs() < max_shift ]

    return frames_df

def prepare_shortvowels(utt_df, utt_names=None, phone_map_file='phones-61-48-39.txt',
                        phn_set=['aa', 'ae', 'ah', 'ao', 'eh', 'ih', 'ix', 'uh']):
    
    ## DATA SELECTION -- utterance based (speed up)
    if utt_names is not None:
        utt_df = utt_df[utt_df['utt_name'].isin(utt_names)]

    ## FEATURE EXTRACTION

    # input feature parameters
    window_nframes = 11
    stride = 2
    shift = 0
    delta, ddelta = False, False
    normMean, normVar = False, True

    # generate dataframe containing frames
    frames_df = assemble_frame_dataframe(utt_df, window_nframes, stride, shift, \
                                        delta, ddelta, normMean, normVar)

    # assemble input feature from window
    features = apply_window(frames_df['data'], frames_df.index, frames_df['window'])
    frames_df['feature'] = reshape_features(features, (-1,)) 

    ## LABELS

    # phone and label mapping
    ocol = 1 # {'61' : 0, '48' : 1, '39' : 2}
    #phn_set =  ['aa', 'ih']#['aa', 'ae', 'ah', 'ao', 'eh', 'ih', 'ix', 'uh'] 
    phn2phn, phn2lab, lab2phn, phn_set = phone_and_label_mapping(phone_map_file, ocol, phn_set)

    # map phones and labels
    frames_df['phn_label'] = frames_df['org_phn_label'].map(phn2phn) # TIMIT phone label -> phone label
    frames_df['int_label'] = frames_df['phn_label'].map(phn2lab)  # phone label ->  integer label

    ## DATA SELECTION -- frame based

    # drop phones not in phn_set
    frames_df = frames_df[~frames_df['int_label'].isna()] 

    # drop phones with large shift in phn_set
    doShift = False
    max_shift = 5 # shift = distance to center frame
    if doShift:  
      frames_df['shift'] = frames_df['frame_idx'] - frames_df['center_frame_idx']
      frames_df = frames_df[frames_df['shift'].abs() < max_shift ]

    return frames_df

def prepare_cnn(utt_df, utt_names=None, phone_map_file='phones-61-48-39.txt'):
    
    ## DATA SELECTION -- utterance based (speed up)
    if utt_names is not None:
        utt_df = utt_df[utt_df['utt_name'].isin(utt_names)]

    ## FEATURE EXTRACTION

    # input feature parameters
    window_nframes = 11
    stride = 2
    shift = 0
    delta, ddelta = False, False
    normMean, normVar = False, True

    # generate dataframe containing frames
    frames_df = assemble_frame_dataframe(utt_df, window_nframes, stride, shift, \
                                        delta, ddelta, normMean, normVar)

    # assemble input feature from window
    features = apply_window(frames_df['data'], frames_df.index, frames_df['window'])
    frames_df['feature'] = reshape_features(features, (-1,)) 

    ## LABELS

    # phone and label mapping
    ocol = 1 # {'61' : 0, '48' : 1, '39' : 2}
    phn_set = None
    phn2phn, phn2lab, lab2phn, phn_set = phone_and_label_mapping(phone_map_file, ocol, phn_set)

    # map phones and labels
    frames_df['phn_label'] = frames_df['org_phn_label'].map(phn2phn) # TIMIT phone label -> phone label
    frames_df['int_label'] = frames_df['phn_label'].map(phn2lab)  # phone label ->  integer label

    ## DATA SELECTION -- frame based

    # drop phones not in phn_set
    frames_df = frames_df[~frames_df['int_label'].isna()] 

    # drop phones with large shift in phn_set
    doShift = False
    max_shift = 5 # shift = distance to center frame
    if doShift:  
      frames_df['shift'] = frames_df['frame_idx'] - frames_df['center_frame_idx']
      frames_df = frames_df[frames_df['shift'].abs() < max_shift ]


## Custom setups

f11s2_setup = {'window_nframes' : 11, 'stride' : 2, 'shift' : 0,
               'delta' : False, 'ddelta' : False,
               'normMean' : False, 'normVar' : True}

f7s2_setup = {'window_nframes' : 7, 'stride' : 2, 'shift' : 0,
               'delta' : False, 'ddelta' : False,
               'normMean' : False, 'normVar' : True}

f7s2d_setup = {'window_nframes' : 7, 'stride' : 1, 'shift' : 0,
               'delta' : True, 'ddelta' : False,
               'normMean' : False, 'normVar' : True}

f13s1_setup = {'window_nframes' : 13, 'stride' : 1, 'shift' : 0,
               'delta' : False, 'ddelta' : False,
               'normMean' : False, 'normVar' : True}

f13s2_setup = {'window_nframes' : 13, 'stride' : 2, 'shift' : 0,
               'delta' : False, 'ddelta' : False,
               'normMean' : False, 'normVar' : True}


def prepare_setup(utt_df, setup, utt_names=None, ocol=1, phn_set=None,  
                  phone_map_file='phones-61-48-39.txt'):
    
    ## DATA SELECTION -- utterance based (speed up)
    if utt_names is not None:
        utt_df = utt_df[utt_df['utt_name'].isin(utt_names)]

    ## FEATURE EXTRACTION

    # input feature parameters
    window_nframes = setup['window_nframes']
    stride = setup['stride']
    shift = setup['shift']
    delta, ddelta = setup['delta'], setup['ddelta']
    normMean, normVar = setup['normMean'], setup['normVar']

    # generate dataframe containing frames
    frames_df = assemble_frame_dataframe(utt_df, window_nframes, stride, shift, \
                                        delta, ddelta, normMean, normVar)

    # assemble input feature from window
    features = apply_window(frames_df['data'], frames_df.index, frames_df['window'])
    frames_df['feature'] = reshape_features(features, (-1,)) 

    ## LABELS

    # phone and label mapping
    phn2phn, phn2lab, lab2phn, phn_set = phone_and_label_mapping(phone_map_file, ocol, phn_set)

    # map phones and labels
    frames_df['phn_label'] = frames_df['org_phn_label'].map(phn2phn) # TIMIT phone label -> phone label
    frames_df['int_label'] = frames_df['phn_label'].map(phn2lab)  # phone label ->  integer label

    ## DATA SELECTION -- frame based

    # drop phones not in phn_set
    frames_df = frames_df[~frames_df['int_label'].isna()] 

    # drop phones with large shift in phn_set
    doShift = False
    max_shift = 5 # shift = distance to center frame
    if doShift:  
      frames_df['shift'] = frames_df['frame_idx'] - frames_df['center_frame_idx']
      frames_df = frames_df[frames_df['shift'].abs() < max_shift ]

    return frames_df

### 2. The TIMIT database 
The experiments in this notebook use the TIMIT database.
Instead of starting from raw speech, we already extracted features (24 filterbank energies) from the speech signal. The feature is obtained by applying the FFT transform (**window step** = 10ms, window size = 30ms, window = Hamming, ...) and mel scaling to the speech waveform (16kHz). Note the window step determines the frame duration (10ms).

The dataframe contains for each utterance:
- utt_name : utterance name
- data : matrix of # frames in utterance x 24 mel-scaled filterbank energies 
- phn_seg : segmentation of utterance into phones
- text: transcription 

The phone mapping translates the 61 phone labels used by TIMIT to a smaller phone set with 48 (or 39) phone labels.
For example, the 61 phone set discriminates between different types of silences (bcl, dcl, epi, h#, ...), whereas the 39 phone set maps all these to a single silence-label (sil).


In [5]:
# download dataframe 
root_url = 'http://homes.esat.kuleuven.be/~spchlab/data/timit/'
utt_df_url = root_url + 'examples/timit-fb24-100Hz-text.pkl' 
utt_df = pd.read_pickle(utt_df_url) 

# download phone mapping
phone_map_file = 'phones-61-48-39.txt'
phone_map_file_url = root_url + 'examples/phones-61-48-39.txt'
write_from_url(phone_map_file_url, phone_map_file)

In [6]:
f11s2_setup = {'window_nframes' : 11, 'stride' : 2, 'shift' : 0,
               'delta' : False, 'ddelta' : False,
               'normMean' : False, 'normVar' : True} 

### DNN vs. GMM on short vowels

Last exercise session on phoneme classification, we compared the performance of DNNs and GMMs. Near the end we remarked the toy problem was too small for meaningful conclusions. Here we compare the two models on a subset of TIMIT, namely the short vowels. 

In [8]:
# subset of phone labels
long_vowels = ['aw', 'ay', 'er', 'ey', 'iy', 'ow', 'oy', 'uw']
short_vowels = ['aa', 'ao', 'ae', 'ah', 'ax', 'eh', 'ih', 'ix', 'uh']

# use a subset of phones
phn_set = short_vowels # short_vowels # long_vowels # None # 

# compute phone and label mapping
ocol = 1 # {0 :'61 phones' : 0, 1 : '48 phones', 2 : '39 phones'}
phn2phn, phn2lab, lab2phn, phn_set = phone_and_label_mapping(phone_map_file, ocol, phn_set)

#### Feature extraction

We use the same feature extraction setup ('f11s2_setup') as above.
Since we are only using the short vowels of TIMIT, examples with other labels are discarted during the feature extraction.

In [9]:
# Prepare train/validation/test set
# ! takes around 2 minutes (~ combining frames in context window)
shortvowels_df = prepare_setup(utt_df, f11s2_setup, phn_set=phn_set)

# phone and label mapping
phn2phn, phn2lab, lab2phn, phn_set = phone_and_label_mapping(phone_map_file, ocol, phn_set)

# split data
train_df, valid_df, test_df = utterance_based_data_split(shortvowels_df)

# input features
dnn_train_X = np.vstack(train_df['feature']).astype('float32') 
dnn_valid_X = np.vstack(valid_df['feature']).astype('float32') 
dnn_test_X = np.vstack(test_df['feature']).astype('float32') 

# labels
train_y = train_df['int_label'].to_numpy().astype('int64') 
valid_y = valid_df['int_label'].to_numpy().astype('int64')
test_y = test_df['int_label'].to_numpy().astype('int64')

#### GMM model

In [11]:
#@title GMM functions

# =============================================================================
# GMM
# =============================================================================

def train_test_GMM(X_train,X_test,y_train,y_test,classes,
                     n_components=1,max_iter=20,tol=1.e-3,print_result=True,print_cmat=False):
    clf_GM = GaussianMixtureClf(classes=classes,n_components=n_components,max_iter=20,tol=1.e-3)
    clf_GM.fit(X_train,y_train)
    y_pred = clf_GM.predict(X_train)
    acc_train = 100.0*skmetrics.accuracy_score(y_train, y_pred)
    y_pred = clf_GM.predict(X_test)
    acc_test = 100.0*skmetrics.accuracy_score(y_test, y_pred) 
    cmat = skmetrics.confusion_matrix(y_test,y_pred)
    if(print_result):
        lls, bics = llscore(clf_GM,X_train,y_train)
        print('Training Set:  Accuracy = %.2f%%     LL = %.2f    BIC = %.2f ' % (acc_train,lls,bics) )
        print('Test Set:      Accuracy = %.2f%%'  % (acc_test) )
    if(print_cmat):
        plot_confusion_matrix(cmat,labels=classes)
    return (acc_test,acc_train)

def llscore(GMM,X,y):
    ''' Average log likelihood per sample over the full data set (X,y) 
    and BIC per sample '''
    ll = 0.
    for k in range(0,GMM.n_classes) :
        ll += GMM.gmm[k].score(X[y== GMM.classes[k],: ])
    lls = ll.mean()
    nparam = ((2*n_dim+1)*n_components -1 ) * GMM.n_classes
    bics = -2*lls + (np.log(X.shape[0])* nparam) / float(X.shape[0])
    return(lls,bics)


##### Input feature: window of frames 

In [13]:
print(dnn_train_X.shape)
print(dnn_test_X.shape)
print(train_y.shape)
print(test_y.shape)
print(dnn_train_X.ndim)
print(dnn_test_X.ndim)
print(train_y.ndim)
print(test_y.ndim)

(215588, 264)
(82256, 264)
(215588,)
(82256,)
2
2
1
1


In [14]:
# GMM model
n_dim = dnn_train_X.shape[1] #  feature dimension
n_components = 5 # n_components to test
classes = phn_set

print("***n_components=%d ***" % (n_components))
_,_ = train_test_GMM(dnn_train_X,dnn_test_X,train_y,test_y,classes,
                     n_components=n_components,print_result=True,print_cmat=True)

***n_components=5 ***


/usr/local/lib/python3.7/dist-packages/pyspch/GaussianMixtureClf.py:90: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  selection = (y== self.classes[k])


ValueError: ignored

##### Input feature: single frame

In [15]:
# single frame features
sframe_train_X = np.vstack(train_df['data']).astype('float32') 
sframe_test_X = np.vstack(test_df['data']).astype('float32') 

In [17]:
# GMM model
n_dim = sframe_train_X.shape[1] #  feature dimension
n_components = 5 #  n_components to test
classes = phn_set

print("***n_components=%d ***" % (n_components))
_,_ = train_test_GMM(sframe_train_X,sframe_test_X,train_y,test_y,classes,
                     n_components=n_components,print_result=True,print_cmat=True)

***n_components=5 ***


/usr/local/lib/python3.7/dist-packages/pyspch/GaussianMixtureClf.py:90: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  selection = (y== self.classes[k])


ValueError: ignored

##### Input feature: MFCC

In [18]:
# MFCC features
mfcc_train_X = dct(sframe_train_X, type=2, axis=1, norm='ortho')
mfcc_test_X = dct(sframe_test_X, type=2, axis=1, norm='ortho')

# add first order temporal derrivatives for GMM
mfcc_train_X = add_derrivatives_data(mfcc_train_X, delta=True, ddelta=False)
mfcc_test_X = add_derrivatives_data(mfcc_test_X, delta=True, ddelta=False)


In [19]:
# GMM model
n_dim = mfcc_train_X.shape[1] #  feature dimension
n_components = 5 #  n_components to test
classes = phn_set

print("***n_components=%d ***" % (n_components))
_,_ = train_test_GMM(mfcc_train_X,mfcc_test_X,train_y,test_y,classes,
                     n_components=n_components,print_result=True,print_cmat=True)

***n_components=5 ***


/usr/local/lib/python3.7/dist-packages/pyspch/GaussianMixtureClf.py:90: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  selection = (y== self.classes[k])


ValueError: ignored